## Import statements for Dataframe manipulation

In [ ]:
import pandas as pd
import numpy as np
import math
from itertools import combinations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/RCV Voting Method Research/Preference Profiles/ResearchProfiles/WithoutWriteins')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create array of preference profiles

In [ ]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]

Test array

In [ ]:
files = ["PierceCounty08CountyExecMemberNoWriteins.csv"]

# Create output file

In [ ]:
output = pd.DataFrame(columns = ["Election", "BCU", "QBC"])

Download completed data

In [ ]:
completedData = pd.read_csv("../SoFar.csv").drop(columns=['Unnamed: 0'])
output = completedData.copy()

# Create dataframe for output

In [ ]:
for f in files:
  df = pd.read_csv(f)
  ranks = df.drop(columns = ['Num. Voters'])
  candidates = ranks.stack().unique()
  for i in range(candidates.shape[0]):
    if candidates[i] == '-':
      candidates = np.delete(candidates, i)
      break

  #don't run if more than 11 candidates
  if candidates.shape[0] > 10:
    continue

  if f in output["Election"].values:
    continue

  allCandidates = candidates.copy()

  data = df.to_numpy()

  print(f)

  row = [f]

  #determine all possible combinations of candidates
  subCandidates = []

  for i in reversed(range(1, candidates.shape[0]-1)):
    for cands in list(combinations(candidates, i)):
      subCandidates.append(cands)

  #determine original winners of complete borda
  bordaWinner = ""
  quadraticWinner = ""

  for m in ["borda", "quadraticComplete"]:
    points = np.full(len(candidates), 0.0)
    points = bordaCompleteEval(m)

    max = 0
    maxIndex = 0

    for i in range(points.shape[0]):
      if points[i] > max:
        max = points[i]
        maxIndex = i

    if m == "borda":
      bordaWinner = candidates[maxIndex]
    elif m == "quadraticComplete":
      quadraticWinner = candidates[maxIndex]

  bordaFail = False
  quadraticFail = False

  for cands in subCandidates:

    if bordaFail and quadraticFail:
      break

    if (bordaWinner in cands) and (quadraticWinner in cands):
      continue

    sdf = df.replace(cands, '-')

    candidates = np.array([i for i in allCandidates if i not in cands])

    data = sdf.to_numpy()

    for i in range(data.shape[0]):
      toChange = False
      for j in range(1, data.shape[1]):
        if toChange:
          if data[i][j] != '-':
            data[i][changeIndex] = data[i][j]
            data[i][j] = '-'
            changeIndex = changeIndex + 1
        else:
          if data[i][j] == '-':
            changeIndex = j
            toChange = True

    #Check BCU
    if (not bordaFail) and (not bordaWinner in cands):
      points = np.full(len(candidates), 0.0)
      points = bordaCompleteEval("borda")

      max = 0
      maxIndex = 0

      for i in range(points.shape[0]):
        if points[i] > max:
          max = points[i]
          maxIndex = i

      if bordaWinner != candidates[maxIndex]:
        bordaFail = True

    #Check QBC
    if (not quadraticFail) and (not quadraticWinner in cands):
      points = np.full(len(candidates), 0.0)
      points = bordaCompleteEval("quadraticComplete")

      max = 0
      maxIndex = 0

      for i in range(points.shape[0]):
        if points[i] > max:
          max = points[i]
          maxIndex = i

      if quadraticWinner != candidates[maxIndex]:
        quadraticFail = True

  row.append("Fail" if bordaFail else "Pass")
  row.append("Fail" if quadraticFail else "Pass")

  output.loc[len(output)] = row

#output file
output.to_csv("../SpoilerRemoveWriteins.csv")

Alaska20POTUS.csv
Berkeley14CityCouncilDist1.csv
Berkeley14CityCouncilDist7.csv
Berkeley14CityCouncilDist8.csv
Berkeley17CityCouncilDist4.csv
Berkeley18CityAuditor.csv
Berkeley18CityCouncilDist1.csv
Berkeley18CityCouncilDist4.csv
Berkeley18CityCouncilDist7.csv
Berkeley18CityCouncilDist8.csv
Berkeley16CityCouncilDist2.csv
Berkeley16CityCouncilDist3.csv
Berkeley16CityCouncilDist5.csv
Berkeley16CityCouncilDist6.csv
Berkeley16Mayor.csv
Bloomington23CityCouncilDist4.csv
Bloomington23SpecialCityCouncilAtLarge.csv
Boulder23Mayor.csv
ElkRidge21Mayor.csv
Hawaii20POTUSCD1CD2.csv
Kansas20POTUS.csv
LasCruces19COUNCILORPOSITION2CITYDIST2COUNCILOR.csv
LasCruces19COUNCILORPOSITION4CITYDIST4COUNCILOR.csv
LasCruces19MAYOR.csv
Maine18DemocraticPrimaryCD2.csv
Maine20DemocraticCandidateStateHouseDistrict41.csv
Maine20DemocraticCandidateStateHouseDist47.csv
Maine20DemocraticCandidateStateHouseDist49.csv
Maine20DemocraticCandidateStateHouseDist90.csv
Maine20DemocraticCandidateStateSenateDist11.csv
Maine20Re

# Borda Methods

Complete Borda

In [ ]:
def bordaCompleteEval(method):

  #function to control which counting method is being used and correct
  def countVotes(rank):
    if method == "divisibleComplete":
      return divisibleBorda(rank - 1)
    elif method == "exponentialComplete":
      return exponentialBorda(rank - 1)
    elif method == "borda":
      return borda(rank - 1)
    elif method  == "quadraticComplete":
      return quadraticBorda(rank - 1)

  #create array of points for each candidate
  points = np.full(len(candidates), 0.0)

  for i in range(data.shape[0]):
    if data[i][1] == '-':
      continue;
    votedCandidates = candidates.copy()
    lastColumn = 1
    for j in range(1, data.shape[1]):
      if data[i][j] == '-':

        #end loop to begin allocating remaining points
        break
      else:
        #allocate votes based on ranking
        for k in range(candidates.shape[0]):
          if data[i][j] == candidates[k]:
            votedCandidates[k] = 'voted'
            points[k] = points[k] + data[i][0]*countVotes(j)
        lastColumn = j + 1

    #calcualate remaining points
    remainder = 0
    for k in range(lastColumn, candidates.shape[0] + 1):
      remainder = remainder + countVotes(k)


    if method != "borda":
      #allocate remaining points equally among remaining candidates
      if candidates.shape[0] - lastColumn + 1 != 0:
        avg = remainder/(float(candidates.shape[0] - lastColumn + 1))
        for k in range(candidates.shape[0]):
          if candidates[k] == votedCandidates[k]:
            points[k] = points[k] + data[i][0]*avg

  return points

Modified Borda

In [ ]:
def mbcBorda():

  for i in range(data.shape[0]):
    if data[i][1] == '-':
      continue;
    votedCandidates = []
    for j in range(1, data.shape[1]):
      if data[i][j] == '-':
        break
      else:
        votedCandidates.append(data[i][j])

    for c in range(len(votedCandidates)):
      for k in range(candidates.shape[0]):
        if votedCandidates[c] == candidates[k]:
          points[k] = points[k] + data[i][0]*(len(votedCandidates) - c)

  return points

Define Borda Count methods

In [ ]:
#Allocate points by index in array via borda count
def borda(rank):
  return candidates.shape[0] - rank

#Allocate points via divisible borda count
def divisibleBorda(rank):
  return 2*borda(rank) - 1

#Allocate points via fibonacci borda count
def fibonacci(n):
  return round(1/math.sqrt(5)*pow((1+math.sqrt(5))/2, n))

def fibonacciBorda(rank):
  return fibonacci(borda(rank) + 1)

#Allocate points via exponential borda count
def exponentialBorda(rank):
  return pow(2, borda(rank) - 1)

def quadraticBorda(rank):
  return borda(rank)*(borda(rank) - 1)/2 + 1

bordaEnum = ["borda", "divisible", "exponential", "fibonacci", "quandratic", "bordaComplete", "divisibleComplete", "exponentialComplete", "fibonacciComplete", "quadraticComplete"]
bordaEnum = enumerate(bordaEnum)

Enter data so far

In [ ]:
output.to_csv("../SoFar.csv")